# 1. 'InferNet(AVG)'
2. 用avg on training set作为testing set的param value值
2. 然后计算testing set的metric

In [19]:
import torch
import numpy as np
import random
import pandas as pd
import os
from tqdm import tqdm
from metric_for_GTs_func import *
from MLP.utils import setup_seed

ARTIFICIAL = True
noise_pct = 0.05
SET_VAL = False
seed = 512      # 3,31,204,223,407,62,508，626 # [4,31,34,204,407,66,508,512]


train_pct = 0.7
vali_pct = 0.2
test_pct = 0.1


if ARTIFICIAL:
    target_path = "../data/artificial_targets_v2_" + "noise=" + str(noise_pct)
    # SA 的 params
    params_opitim_path_SA = "../data/SA_PT/params_opitim_artificial_v2_noise_"+str(noise_pct)+".csv"
    # params_opitim_path_SA = "../data/SA_PT/params_opitim_artificial_noise_"+str(noise_pct)+".csv"
    params_opitim_unified_path = r"../data/SA_PT/params_artificial_unified_noise="+str(noise_pct)+ "_seed="+str(seed)+".csv"
else:
    target_path = "../data/targets_all"
    # SA 的 params
    params_opitim_path_SA = "../data/SA_PT/params_opitim_delta_T.csv"
    params_opitim_unified_path = "../data/SA_PT/params_opitim_unified_seed="+str(seed)+".csv"

# Small dataset
small_settings_NN_path = r"../data/small_settings_NN.csv"
# Large data
large_settings_NN_path = r'E:\DATA\large_dta\large_settings_NN.csv'


In [18]:
AVG = False
# 计算GT2-AVG
UNIFIED = True  # 计算GT2-Unified

In [20]:
# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)

data_key_small = pd.read_csv(small_settings_NN_path,encoding="utf-8")
data_key_large = pd.read_csv(large_settings_NN_path,encoding="utf-8")

data_key = pd.concat([data_key_small,data_key_large],axis=0,ignore_index=True)

In [21]:
setup_seed(seed)

DATA_len = len(target_all_path)
shuffled_indices = np.random.permutation(DATA_len)


train_idx = shuffled_indices[:int(train_pct * DATA_len)]
if SET_VAL:
    tmp = int((train_pct + vali_pct) * DATA_len)
    val_idx = shuffled_indices[int(train_pct * DATA_len):tmp]
    test_idx = shuffled_indices[tmp:]
else :
    # Exchange. 20% for testing
    tmp = int((train_pct + vali_pct) * DATA_len)
    test_idx = shuffled_indices[int(train_pct * DATA_len):tmp]
    val_idx = shuffled_indices[tmp:]
print(f"test set pct: {len(test_idx)/DATA_len}")

test set pct: 0.19984326018808776


-
- 计算avg param values
- 或者计算unified

In [22]:
import numpy as np

# 创建原始数组
arr = np.array([1, 2, 3.5, 4.5])

# 创建布尔数组
mask = arr > 2.5

# 使用布尔数组提取大于2.5的数值
result = arr[mask]
arr[arr > 2.5]

array([3.5, 4.5])

In [23]:
if AVG:
    params_opitim_SA = pd.read_csv(params_opitim_path_SA,encoding="utf-8")
    params_train = params_opitim_SA.iloc[train_idx,:].copy()
    params_avg = params_train.reset_index().mean()

    alpha = params_avg.alpha
    labda = params_avg.labda
elif UNIFIED:
    params_train = pd.read_csv(params_opitim_unified_path,encoding="utf-8")
    # avg_loss最低的那个
    idx = params_train.avg_loss.idxmin(0)
    params_unified = params_train.iloc[idx,:]

    alpha = params_unified.alpha
    labda = params_unified.labda

- 计算testing上的metric

In [24]:
NLL_metric = []
alpha,labda,seed

(-0.000597471772696, 9.17836246339773, 512)

In [25]:
for i in test_idx:
        # Get target data
        target_path_i_path = os.path.join(target_path, target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
        target = list(target_df.iloc[:,0])

        # Get the auction setting
        settings_df = data_key.iloc[i,:]

        # GT-2: SA-AVG
        NLL_metric.append(GT_2_uniq(settings_df,target,alpha,labda))


In [26]:
len(NLL_metric)

255

In [27]:
print("seed=",seed)
np.mean(NLL_metric)

seed= 512


6.425299187556018